## Alternative Process Plan Scheduling

This notebook demonstrates how to model and solve the classical Resource-Constrained Project Scheduling Problem with Alternative Process Plan
using Constraint Programming with IBM’s CP Optimizer via the [`docplex.cp`](https://ibmdecisionoptimization.github.io/docplex-doc/cp/refman.html) Python API.

### Problem Definition

### CPLEX Formulation

$$
\begin{aligned}
\min \quad
& \mathrm{endOf}(\mathrm{x}_n)
\qquad &\qquad & \text{(1)} \\[2mm]
\text{s.t.} \quad
& \mathrm{presenceOf}(\mathrm{x}_{1}) = 1,
\qquad &\qquad & \text{(2)} \\[1mm]
& \mathrm{if\_then}\!\left(\mathrm{presenceOf}(\mathrm{x}_i) \wedge \mathrm{presenceOf}(\mathrm{x}_j),\; \mathrm{endBeforeStart}(\mathrm{x}_i, \mathrm{x}_j)\right),
\qquad & \forall (i,j)\in A
\quad & \text{(3)} \\[1mm]
& \sum_{k \in K_{p_l}} \mathrm{presenceOf}(\mathrm{x}_{b_k}) = \mathrm{presenceOf}(\mathrm{x}_{p_l}),
\qquad & \forall l \in L
\quad & \text{(4)} \\[1mm]
& \mathrm{presenceOf}(\mathrm{x}_i) = \left( \sum_{k \in B_i} \mathrm{presenceOf}(\mathrm{x}_{b_k}) > 0 \right),
\qquad & \forall i \in N \setminus \{1\}
\quad & \text{(5)} \\[1mm]
& \sum_{i \in N} \mathrm{pulse}(\mathrm{x}_i, r_{i,v}) \le a_v,
\qquad & \forall v \in R
\quad & \text{(6)} \\[1mm]
& \text{interval } \mathrm{x}_i\ \text{optional, size}=d_i,
\qquad & \forall i \in N
\quad & \text{(7)}
\end{aligned}
$$

**Objective:**

* **(1)** Minimize the **makespan** $C_{\max}$, defined as the end time of the project's single sink activity $n$.

**Modeling Constraints:**

* **(2)** Ensures the project's single source activity (activity 1) is always selected ("present").
* **(3)** Enforces all precedence relations $(i,j) \in A$. The `endBeforeStart` constraint is enforced conditionally, only if *both* activities $i$ and $j$ are present in the schedule.
* **(4)** Enforces subgraph branch selection. For each alternative subgraph $l$, exactly one branch (represented by a branching activity $b_k$) must be selected if the principal activity $p_l$ is present.
* **(5)** Enforces activity selection. An activity $i$ is present if and only if at least one (a disjunction, or OR) of the branches $k$ it belongs to is selected.
* **(6)** Enforces renewable resource limits. The `pulse` function models the resource demand $r_{i,v}$ during the execution of a present interval. The sum of all pulses on a resource $v$ must never exceed its capacity $a_v$.

**Variables:**

* **(7)** $\mathrm{x}_i$: An optional interval variable for each activity $i \in N$. Its presence is controlled by a boolean variable and it has a fixed duration $d_i$.

#### Symbols and Notation

| Symbol / Function | Meaning | $\text{docplex.cp}$ Reference |
| :--- | :--- | :--- |
| $N$ | Set of topologically ordered activities |  |
| $n$ | The total number of activities (sink activity index) |  |
| $A$ | Set of end-start precedence constraints |  |
| $L$ | Set of alternative subgraphs |  |
| $R$ | Set of renewable resources |  |
| $d_i$ | Predetermined duration of activity $i$ |  |
| $r_{i,v}$ | Demand of activity $i$ for renewable resource $v$ |  |
| $a_v$ | Capacity of renewable resource $v$ |  |
| $p_l$ | Principal activity of alternative subgraph $l$ |  |
| $b_k$ | Branching activity of alternative branch $k$ |  |
| $K_{p_l}$ | Set of alternative branches in subgraph $l$ |  |
| $B_i$ | Set of branches activity $i$ belongs to |  |
| $\mathrm{x}_i$ | Optional interval variable for activity $i$ | [docplex.cp.modeler.interval\_var](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.interval_var) |
| $\mathrm{presenceOf}(\mathrm{x}_i)$ | $1$ if the optional interval $\text{x}_{i}$ (activity $i$) is present, else $0$ | [docplex.cp.modeler.presence\_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.presence_of) |
| $\mathrm{endOf}(\mathrm{x}_n)$ | End time of interval $\text{x}_{n}$ | [docplex.cp.modeler.end\_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_of) |
| $\min \mathrm{endOf}(\dots)$ | Objective: minimize makespan | [docplex.cp.modeler.minimize](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.minimize) |
| $\mathrm{endBeforeStart}(\dots)$ | Non-overlapping precedence constraint | [docplex.cp.modeler.end\_before\_start](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_before_start) |
| $\mathrm{if\_then}(\dots)$ | Logical implication constraint | [docplex.cp.modeler.if\_then](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.if_then) |
| $\mathrm{pulse}(\mathrm{x}_i, r_{i,v})$ | Time-varying usage (height $r_{i,v}$ when $\text{x}_{i}$ runs) | [docplex.cp.modeler.pulse](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.pulse) |
| $\sum \mathrm{pulse}(\dots) \le a_v$ | Cumulative resource limit | (Sum of `pulse` expressions) |